# Run pipeline (or its steps) on dev data and obtain METEOR-0.25 score 

In [ ]:
!echo $OPENAI_API_KEY

## ⚙️ 0. Prep environment 

In [ ]:
!python -m nltk.downloader punkt


In [12]:
%pip install -r requirements.txt
python -m spacy download en_core_web_lg
python -m nltk.downloader punkt
python -m nltk.downloader wordnet
!sh script/copy_data.sh

/home/ullriher/venvs/2023feb/lib/python3.10/site-packages/thinc/compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
/home/ullriher/venvs/2023feb/lib/python3.10/site-packages/thinc/compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]
2024-07-01 13:21:36.239112: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /mnt/appl/software/Python/3.10.4-GCCcore-11.3.0-bare/lib:/mnt/appl/software/OpenSSL/1.1/lib:/mnt/appl/software/libffi/3.4.2-GCCcore-11.3.0/lib64:/mnt/appl/software/XZ/5.2.5-GCCcore-11.3.0/lib:/mnt/appl/software/SQLite/3.38.3-GCCcore-11.3.0/lib:/mnt/appl/software/Tcl/8.6.12-GCCcore-11.3.0/lib:/mnt/appl/software/libreadline/8.1.2-GCCcore-11.3.0/li

^C


## 🔎 1. Get precomputed Google API results and Scrape text from their URLs

In [ ]:
!bash script/scraper.sh dev 0 500

## 🥇 2. Rank Search results with BM25

In [ ]:
%run src/reranking/bm25_sentences.py

## 💬  3. Generate QA pair for the top sentences

In [ ]:
%run src/reranking/question_generation_top_sentences.py

## 🥈 4. Rerank the QA pairs

In [ ]:
%run src/reranking/rerank_questions.py

## 🤥 5. Predict veracity

In [ ]:
%run src/prediction/veracity_prediction.py

## 📊 6. Evaluate the pipeline

In [9]:
%run src/prediction/evaluate_veracity.py

Question-only score (HU-meteor):             0.9985645061577254
Question-answer score (HU-meteor):           0.8646194096208928
Veracity F1 scores:
 * Supported:                                0.9262295081967213
 * Refuted:                                  0.9028006589785833
 * Not Enough Evidence:                      0.6213592233009708
 * Conflicting Evidence/Cherrypicking:       0.2608695652173913
 * macro:                                    0.6778147389234167
 * acc:                                      0.85
--------------------
AVeriTeC scores:
 * Veracity scores (meteor @ 0.1):           0.85
 * Veracity scores (meteor @ 0.2):           0.834
 * Veracity scores (meteor @ 0.25):          0.828
 * Veracity scores (meteor @ 0.3):           0.82
 * Veracity scores (meteor @ 0.4):           0.804
 * Veracity scores (meteor @ 0.5):           0.778
--------------------
AVeriTeC scores by type @ 0.25:
 * Veracity scores (Event/Property Claim):   0.8413429830058025
 * Veracity scores (Pos